In [1]:
import pandas as pd
import pickle
import datetime as dt
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 5000)

In [2]:
#from LogIn import LogIn
#kite = LogIn().return_kite_obj()

In [3]:
budget_df = pd.read_csv('budget.csv')

In [4]:
budget_df

,instrument,budget,return
0,NSE:UNIONBANK,400,0.02
1,NSE:DCBBANK,400,0.02
2,NSE:YESBANK,400,0.02
3,NSE:WIPRO,400,0.02
4,NSE:TATASTEEL,400,0.02
5,NSE:SBIN,400,0.02
6,NSE:PNB,400,0.02
7,NSE:ONGC,400,0.02
8,NSE:OIL,400,0.02
9,NSE:NTPC,400,0.02


# Inputs

In [5]:
from_year = 2015
to_year = 2018

In [6]:
leverage = 15
exit_time = dt.time(15, 10)

# Process

In [7]:
# Download data of all companies
download = False

if download:
    
    for year in range(from_year, to_year+1):
        
        from_date = '{}-01-01'.format(year)
        to_date = '{}-12-31'.format(year)
        print(from_date, to_date)
        for instrument in budget_df.instrument.tolist():

            try:
                print('------------')
                print(instrument)
                instrument_token = kite.ohlc(instrument)[instrument]['instrument_token']
                data = kite.historical_data(instrument_token=instrument_token, from_date=from_date, to_date=to_date, interval='minute', continuous=False)
                data_df = pd.DataFrame(data)
                data_df.to_pickle('./data/{}_{}_{}.pkl'.format(instrument, from_date, to_date))

            except Exception as ex:
                print('Failed for {} : {}'.format(instrument, ex))

In [8]:
def get_clean_data(instrument, from_year, to_year):
    
    print(instrument)
    
    all_data_df = pd.DataFrame()
    
    for year in range(from_year, to_year+1):
        
        #print(year)
        
        from_date = '{}-01-01'.format(year)
        to_date = '{}-12-31'.format(year)
        
        data_df = pd.read_pickle('./data/{}_{}_{}.pkl'.format(instrument, from_date, to_date))
        data_df.rename(columns={'date':'timestamp'}, inplace=True)
        data_df['date'] = data_df['timestamp'].dt.date
        data_df['day'] = data_df['timestamp'].dt.day_name()
        #print(len(data_df))

        sanity_check = data_df.groupby('date').count().reset_index()
        incomplete_data_days = sanity_check[sanity_check['volume']<300]
        #print(incomplete_data_days)

        data_df = data_df[~data_df['date'].isin(incomplete_data_days.date.tolist())]
        data_df['instrument'] = instrument
        #print(len(data_df)/375)
        
        all_data_df = all_data_df.append(data_df, ignore_index=True)
        
    return all_data_df

In [9]:
header_list = ['instrument', 'high_prev', 'low_prev', 'open_today', 'condition', 'transaction_type', 'trade_number', 'level', 'budget', 'return', 'daily_khwab', 'actual_khwab', 'quantity', 'budget_required', 'price', 'trigger_price', 'squareoff', 'stoploss', 'order_id', 'timestamp', 'order_type', 'status', 'profit', 'pl_tag', 'flag', 'adhoora_khwab']


In [10]:
for instrument in budget_df.instrument.tolist():
    try:
        d = get_clean_data(instrument, from_year, to_year)
        print(len(d.date.unique()))
    except:
        print('error', instrument)

NSE:UNIONBANK
962
NSE:DCBBANK
962
NSE:YESBANK
962
NSE:WIPRO
962
NSE:TATASTEEL
962
NSE:SBIN
962
NSE:PNB
962
NSE:ONGC
962
NSE:OIL
962
NSE:NTPC
962
NSE:BEL
962
NSE:BIOCON
962
NSE:CANBK
962
NSE:CASTROLIND
962
NSE:CHOLAFIN
962
NSE:COALINDIA
962
NSE:ENGINERSIN
962
NSE:EXIDEIND
962
NSE:JSWSTEEL
962
NSE:MANAPPURAM
962
NSE:SAIL
962
NSE:RECLTD
962
NSE:NMDC
962
NSE:MFSL
952
NSE:IDBI
962
NSE:ARVIND
962
NSE:BHEL
962
NSE:TATAMTRDVR
962
NSE:TATAPOWER
962
NSE:ADANIPOWER
962
NSE:BSOFT
962
NSE:ASHOKLEY
962
NSE:NCC
962
NSE:BANKINDIA
962
NSE:FEDERALBNK
962
NSE:MOTHERSUMI
962
NSE:BANKBARODA
962
NSE:JINDALSTEL
962
NSE:L&TFH
962
NSE:PFC
962
NSE:EQUITAS
error NSE:EQUITAS
NSE:TATAMOTORS
962
NSE:IOC
962
NSE:ADANIENT
962
NSE:GAIL
962
NSE:VEDL
962
NSE:APOLLOTYRE
962
NSE:DLF
962
NSE:HINDALCO
962
NSE:AMBUJACEM
962
NSE:HINDZINC
962
NSE:CADILAHC
962
NSE:POWERGRID
962
NSE:HINDPETRO
962
NSE:INFRATEL
962
NSE:ITC
962
NSE:PETRONET
962
NSE:TATAGLOBAL
962
NSE:UJJIVAN
error NSE:UJJIVAN
NSE:TORNTPOWER
962
NSE:IGL
962
NSE:ADAN

In [ ]:
calculate = True

if calculate:
    
    all_trades = pd.DataFrame(columns=header_list)
    
    for i in range(len(d.date.unique().tolist())-1):
        
        trades_today = pd.DataFrame(columns=header_list)
        
        for instrument in budget_df.instrument.tolist():

            try:
                all_data_df = get_clean_data(instrument, from_year, to_year)

                date_prev = all_data_df.date.unique().tolist()[i]
                data_prev = all_data_df[all_data_df['date']==date_prev]
                open_prev = data_prev.open.iloc[0]
                close_prev = data_prev.open.iloc[-1]
                high_prev = data_prev.high.max()
                low_prev = data_prev.low.min()

                date_today = all_data_df.date.unique().tolist()[i+1]
                data_today = all_data_df[all_data_df['date']==date_today]
                open_today = data_today.open.iloc[0]

                if (open_today > high_prev) & (open_today < 1.15 * high_prev):

                    condition = 'high'
                    transaction_type = 'sell'

                elif (open_today < low_prev) & (open_today > 0.85 * low_prev):

                    condition = 'low'
                    transaction_type = 'buy'

                else:

                    condition = None
                    transaction_type = None
                
                if condition is not None:
                    trades_today = trades_today.append(pd.DataFrame(data={'instrument':[instrument], 
                                                      'high_prev':[high_prev], 
                                                      'low_prev':[low_prev], 
                                                      'open_today':[open_today], 
                                                      'condition':[condition], 
                                                      'transaction_type':[transaction_type]}), ignore_index=True)

            except Exception as ex:
                print(instrument, ex)
                                                                                      
#         for instrument in trades_today.instrument.tolist():
            
#             try:
                
#                 # Trim beyond exit time
#                 data_today = data_today[data_today['timestamp'].dt.time<exit_time]
#                 all_data_df = get_clean_data(instrument, from_year, to_year)

#                 date_prev = all_data_df.date.unique().tolist()[i]
#                 data_prev = all_data_df[all_data_df['date']==date_prev]
#                 open_prev = data_prev.open.iloc[0]
#                 close_prev = data_prev.open.iloc[-1]
#                 high_prev = data_prev.high.max()
#                 low_prev = data_prev.low.min()

#                 date_today = all_data_df.date.unique().tolist()[i+1]
#                 data_today = all_data_df[all_data_df['date']==date_today]

#                 # Trim beyond exit time
#                 data_today = data_today[data_today['timestamp'].dt.time<exit_time]
#                 open_today = data_today.open.iloc[0]

#                 if (open_today > high_prev) & (open_today < 1.15 * high_prev):

#                     condition = 'high'
#                     transaction_type = 'sell'

#                 elif (open_today < low_prev) & (open_today > 0.85 * low_prev):

#                     condition = 'low'
#                     transaction_type = 'buy'

#                 else:

#                     condition = None
#                     transaction_type = None
                
#                 if condition is not None:

                            
                            
#                     valid_trades = all_trades[(all_trades['instrument']==instrument) & (all_trades['condition']==condition) & (all_trades['status']=='complete')]
#                     last_valid_trade = valid_trades[valid_trades['trade_number']==valid_trades['trade_number'].max()]

#                     # Calculate trade number
#                     if len(last_valid_trade):
#                         trade_number = int(last_valid_trade['trade_number']) + 1
#                     else:
#                         trade_number = 1
                    
#                     # Calculate level
#                     if len(last_valid_trade):
#                         if float(last_valid_trade['pl_tag'])==1.0:
#                             level = 1
#                         else:
#                             level = float(last_valid_trade['level']) + 1
#                     else:
#                         level = 1

#                     # Calculate budget and return
#                     if level==1:
#                         budget = float(budget_df[budget_df['instrument']==instrument]['budget'])
#                         return_ = float(budget_df[budget_df['instrument']==instrument]['return'])
#                     else:
#                         budget = float(last_valid_trade['budget'])
#                         return_ = float(last_valid_trade['return'])

#                     # Calculate daily khwab
#                     if len(last_valid_trade):
#                         if int(last_valid_trade['flag'])==1:
#                             daily_khwab = round(return_ * budget, 2)
#                         else:
#                             daily_khwab = 0
#                     else:
#                         daily_khwab = round(return_ * budget, 2)

#                     # Calculate actual khwab
#                     if len(last_valid_trade):
#                         actual_khwab = max(round(float(last_valid_trade['adhoora_khwab']) + daily_khwab, 2), round(return_ * budget, 2))
#                     else:
#                         actual_khwab = round(return_ * budget, 2)

#                     # Caluclate tax
#                     tax = (0.0005647 * actual_khwab)/return_
#                     actual_khwab = round(actual_khwab + tax, 2)
                    
#                     # Calculate quantity
#                     quantity = int(round(actual_khwab/(return_ * open_today)))

#                     # Calculate budget required
#                     budget_required = round(quantity * open_today, 2)

#                     # Calculate price and trigger price
#                     if transaction_type=='buy':
#                         price = round(open_today + 0.05 * max(round(20 * 0.0010 * open_today), 2), 2)
#                         trigger_price = round(open_today + 0.05 * max(round(20 * 0.0005 * open_today),1), 2)
#                     else:
#                         price = round(open_today - 0.05 * max(round(20 * 0.0010 * open_today), 2), 2)
#                         trigger_price = round(open_today - 0.05 * max(round(20 * 0.0005 * open_today),1), 2)

#                     # Calculate square off and stoploss
#                     squareoff = round(return_ * price, 1)
#                     stoploss = round(return_ * price, 1)

#                     # Calculate order_id, order_type
#                     order_id = trade_number
#                     order_type = 'SL'

#                     # Calculate timestamp, status and profit
#                     if transaction_type=='buy':
                        
#                         if data_today.high.max() < trigger_price:

#                             timestamp = data_today.timestamp.iloc[0]
#                             status = 'incomplete'
#                             profit = 0

#                         else:

#                             status = 'complete'
#                             for index, row in data_today.iterrows():

#                                 if row['high'] > price + squareoff:
#                                     timestamp = row['timestamp']
#                                     profit = round(squareoff * quantity, 2)
#                                     break

#                                 elif row['low'] < price - stoploss:
#                                     timestamp = row['timestamp']
#                                     profit = round(-1 * stoploss * quantity, 2)
#                                     break

#                                 else:
#                                     timestamp = row['timestamp']
#                                     profit = round((row['close']-price) * quantity, 2)

#                     else:

#                         if data_today.low.min() > trigger_price:
#                             timestamp = data_today.timestamp.iloc[0]
#                             status = 'incomplete'
#                             profit = 0        

#                         else:

#                             status = 'complete'
#                             for index, row in data_today.iterrows():

#                                 if row['low'] < price - squareoff:
#                                     timestamp = row['timestamp']
#                                     profit = round(squareoff * quantity, 2)
#                                     break

#                                 elif row['high'] > price + stoploss:
#                                     timestamp = row['timestamp']
#                                     profit = round(-1 * stoploss * quantity, 2)
#                                     break

#                                 else:
#                                     timestamp = row['timestamp']
#                                     profit = round((price - row['close']) * quantity, 2)

#                     # Calculate P/L tag
#                     pl_tag = round(round(float(profit), 2) / (float(squareoff) * float(quantity)), 1)

#                     # Calculate flag
#                     if abs(float(pl_tag))==1:
#                         flag = 1
#                     else:
#                         flag = 0

#                     # Calculate adhoora khwab
#                     adhoora_khwab = round(float(actual_khwab) - profit, 2)

#                     trades_today = pd.DataFrame(data={'instrument':[instrument], 
#                                                       'high_prev':[high_prev], 
#                                                       'low_prev':[low_prev], 
#                                                       'open_today':[open_today], 
#                                                       'condition':[condition], 
#                                                       'transaction_type':[transaction_type], 
#                                                       'trade_number':[trade_number], 
#                                                       'level':[level], 
#                                                       'budget':[budget], 
#                                                       'return':[return_], 
#                                                       'daily_khwab':[daily_khwab], 
#                                                       'actual_khwab':[actual_khwab], 
#                                                       'quantity':[quantity], 
#                                                       'budget_required':[budget_required], 
#                                                       'price':[price], 
#                                                       'trigger_price':[trigger_price], 
#                                                       'squareoff':[squareoff], 
#                                                       'stoploss':[stoploss], 
#                                                       'order_id':[order_id], 
#                                                       'timestamp':[timestamp], 
#                                                       'order_type':[order_type], 
#                                                       'status':[status], 
#                                                       'profit':[profit], 
#                                                       'pl_tag':[pl_tag], 
#                                                       'flag':[flag], 
#                                                       'adhoora_khwab':[adhoora_khwab]
#                                                      })

#                     all_trades = all_trades.append(trades_today, ignore_index = True)
#         except Exception as ex:
#             print('Failed calculation for {} : {}'.format(instrument, ex))
            
#     all_trades.to_pickle('all_trades_opp_brokerage_{}_{}.pkl'.format(from_year, to_year))
    
# else:
    
#     all_trades = pd.read_pickle('all_trades_opp_brokerage_{}_{}.pkl'.format(from_year, to_year))

NSE:UNIONBANK
NSE:DCBBANK


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


NSE:YESBANK
NSE:WIPRO
NSE:TATASTEEL
NSE:SBIN
NSE:PNB
NSE:ONGC
NSE:OIL
NSE:NTPC
NSE:BEL
NSE:BIOCON
NSE:CANBK


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


NSE:CASTROLIND
NSE:CHOLAFIN
NSE:COALINDIA
NSE:ENGINERSIN
NSE:EXIDEIND
NSE:JSWSTEEL
NSE:MANAPPURAM


...

NSE:ZEEL
NSE:GLENMARK
NSE:GLENMARK 'DataFrame' object has no attribute 'dt'
NSE:BHARTIARTL
NSE:TVSMOTOR
NSE:HEXAWARE
NSE:ICICIPRULI
NSE:ICICIPRULI 'timestamp'
NSE:UNIONBANK
NSE:DCBBANK
NSE:YESBANK
NSE:WIPRO
NSE:TATASTEEL
NSE:SBIN
NSE:PNB
NSE:ONGC
NSE:OIL
NSE:NTPC
NSE:BEL
NSE:BIOCON
NSE:CANBK
NSE:CASTROLIND
NSE:CHOLAFIN
NSE:COALINDIA
NSE:ENGINERSIN
NSE:EXIDEIND
NSE:JSWSTEEL
NSE:MANAPPURAM
NSE:SAIL
NSE:RECLTD
NSE:NMDC
NSE:MFSL
NSE:IDBI
NSE:ARVIND
NSE:BHEL
NSE:TATAMTRDVR
NSE:TATAPOWER
NSE:ADANIPOWER
NSE:BSOFT
NSE:ASHOKLEY
NSE:NCC
NSE:BANKINDIA
NSE:FEDERALBNK
NSE:MOTHERSUMI
NSE:BANKBARODA
NSE:JINDALSTEL
NSE:L&TFH
NSE:PFC
NSE:EQUITAS
NSE:EQUITAS 'timestamp'
NSE:TATAMOTORS
NSE:IOC


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


NSE:ADANIENT
NSE:GAIL
NSE:VEDL
NSE:APOLLOTYRE


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


NSE:DLF
NSE:HINDALCO
NSE:AMBUJACEM
NSE:HINDZINC
NSE:CADILAHC
NSE:POWERGRID
NSE:HINDPETRO
NSE:INFRATEL
NSE:ITC
NSE:PETRONET
NSE:TATAGLOBAL
NSE:UJJIVAN
NSE:UJJIVAN 'timestamp'
NSE:TORNTPOWER
NSE:IGL
NSE:ADANIPORTS
NSE:BERGEPAINT
NSE:BPCL
NSE:ZEEL
NSE:GLENMARK
NSE:GLENMARK 'DataFrame' object has no attribute 'dt'
NSE:BHARTIARTL
NSE:TVSMOTOR
NSE:HEXAWARE
NSE:ICICIPRULI
NSE:ICICIPRULI 'timestamp'
NSE:UNIONBANK
NSE:DCBBANK
NSE:YESBANK
NSE:WIPRO
NSE:TATASTEEL


# High Trades Analysis

In [ ]:
high_trades = all_trades[(all_trades['condition']=='high') & (all_trades['status']=='complete')]

In [ ]:
high_trades_loop = pd.DataFrame()
for instrument in high_trades.instrument.unique().tolist():
    instrument_high_trades = high_trades[high_trades['instrument']==instrument]
    instrument_high_trades_loop = instrument_high_trades[instrument_high_trades['trade_number']<=instrument_high_trades[instrument_high_trades['pl_tag']==1.0].trade_number.max()]
    high_trades_loop = high_trades_loop.append(instrument_high_trades_loop, ignore_index=True)
    

In [ ]:
high_trades

# Low Trades Analysis

In [ ]:
low_trades = all_trades[(all_trades['condition']=='low') & (all_trades['status']=='complete')]

In [ ]:
low_trades_loop = pd.DataFrame()
for instrument in low_trades.instrument.unique().tolist():
    instrument_low_trades = low_trades[low_trades['instrument']==instrument]
    instrument_low_trades_loop = instrument_low_trades[instrument_low_trades['trade_number']<=instrument_low_trades[instrument_low_trades['pl_tag']==1.0].trade_number.max()]
    low_trades_loop = low_trades_loop.append(instrument_low_trades_loop, ignore_index=True)
    

In [ ]:
low_trades.head()

# Results

In [ ]:
pd.DataFrame(data={'high':[len(high_trades),
                           len(high_trades_loop),
                           round(high_trades_loop.profit.sum(), 2), 
                           round(high_trades_loop.profit.sum()/budget_df.budget.sum() * 100, 2),
                           round(high_trades_loop.budget_required.max()), 
                           round(high_trades_loop.budget_required.max()/leverage)],
                   'low':[len(low_trades),
                          len(low_trades_loop),
                          round(low_trades_loop.profit.sum(), 2), 
                          round(low_trades_loop.profit.sum()/budget_df.budget.sum() * 100, 2),
                          round(low_trades_loop.budget_required.max()),
                          round(low_trades_loop.budget_required.max()/leverage)]
                   }, 
             index = ['total_trades', 'trades_considered', 'profit', 'perc_return', 'hightest_budget_required', 'hightest_margin_required'])

# Daily Margin Required Analysis

In [ ]:
all_trades['date'] = all_trades['timestamp'].dt.date
all_trades['margin_required'] = all_trades['budget_required']/leverage
margins = all_trades.groupby('date').agg({'budget_required':sum, 'margin_required':sum, 'instrument':list}).reset_index()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def stacked_bar(data, series_labels, category_labels=None, 
                show_values=False, value_format="{}", y_label=None, 
                grid=True, reverse=False):
    """Plots a stacked bar chart with the data and labels provided.

    Keyword arguments:
    data            -- 2-dimensional numpy array or nested list
                       containing data for each series in rows
    series_labels   -- list of series labels (these appear in
                       the legend)
    category_labels -- list of category labels (these appear
                       on the x-axis)
    show_values     -- If True then numeric value labels will 
                       be shown on each bar
    value_format    -- Format string for numeric value labels
                       (default is "{}")
    y_label         -- Label for y-axis (str)
    grid            -- If True display grid
    reverse         -- If True reverse the order that the
                       series are displayed (left-to-right
                       or right-to-left)
    """

    ny = len(data[0])
    ind = list(range(ny))

    axes = []
    cum_size = np.zeros(ny)

    data = np.array(data)

    if reverse:
        data = np.flip(data, axis=1)
        category_labels = reversed(category_labels)

    for i, row_data in enumerate(data):
        axes.append(plt.bar(ind, row_data, bottom=cum_size, 
                            label=series_labels[i]))
        cum_size += row_data

    if category_labels:
        plt.xticks(ind, category_labels, rotation='vertical')

    if y_label:
        plt.ylabel(y_label)

    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    if grid:
        plt.grid()

    if show_values:
        for axis in axes:
            for bar in axis:
                w, h = bar.get_width(), bar.get_height()
                plt.text(bar.get_x() + w/2, bar.get_y() + h/2, 
                         value_format.format(h), ha="center", 
                         va="center")

In [ ]:
df2 = all_trades.groupby(['date', 'instrument'])['margin_required'].sum().unstack('instrument').fillna(0)

df = df2.transpose()

plt.figure(figsize=(50, 10))

series_labels = df.index.tolist()

data = df.values

category_labels = df.columns.tolist()

stacked_bar(
    data, 
    series_labels, 
    category_labels=category_labels, 
    show_values=False,
    value_format="{:.1f}",
    y_label="Total margin required"
)

plt.savefig('margin_required_opp_brokerage_{}_{}.png'.format(from_year, to_year))
plt.show()

## Brokerage Calculation

In [ ]:
# brokerage = min(40, 0.0002 * budget_required)
# stt = 0.00025 * budget_required
# transaction_charges = 2 * 0.0000325 * budget_required
# gst = 0.18 * (brokerage + transaction_charges)
# sebi = 2 * 10 * budget_required/10000000
# total_tax_and_charges = brokerage + stt + transaction_charges + gst + sebi

# profit = profit - total_tax_and_charges